In [1]:
# inspired by https://www.kaggle.com/nickycan/lb-0-3805009-python-edition
import numpy as np   ## linear algebra
import pandas as pd  ## data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split  ## split train and test
from IPython.display import display ## Allows the use of display() for DataFrames

import matplotlib.pyplot as plt  ## data visualization

import gc   ## clean up 
import time ## time it

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

aisles.csv
departments.csv
order_products__prior.csv
order_products__train.csv
orders.csv
products.csv
sample_submission.csv



In [2]:
def load_data(path_data):
    '''
    --------------------------------order_product--------------------------------
    * Unique in order_id + product_id
    '''
    priors = pd.read_csv(path_data + 'order_products__prior.csv', 
                     dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    train = pd.read_csv(path_data + 'order_products__train.csv', 
                    dtype={
                            'order_id': np.int32,
                            'product_id': np.uint16,
                            'add_to_cart_order': np.int16,
                            'reordered': np.int8})
    '''
    --------------------------------order--------------------------------
    * This file tells us which set (prior, train, test) an order belongs
    * Unique in order_id
    * order_id in train, prior, test has no intersection
    * this is the #order_number order of this user
    '''
    orders = pd.read_csv(path_data + 'orders.csv', 
                         dtype={
                                'order_id': np.int32,
                                'user_id': np.int64,
                                'eval_set': 'category',
                                'order_number': np.int16,
                                'order_dow': np.int8,
                                'order_hour_of_day': np.int8,
                                'days_since_prior_order': np.float32})
    '''
    --------------------------------product--------------------------------
    * Unique in product_id
    '''
    products = pd.read_csv(path_data + 'products.csv')
    aisles = pd.read_csv(path_data + "aisles.csv")
    departments = pd.read_csv(path_data + "departments.csv")
    sample_submission = pd.read_csv(path_data + "sample_submission.csv")
    
    return priors, train, orders, products, aisles, departments, sample_submission


In [3]:
path_data = './data/'
df_priors, df_train, df_orders, df_products, df_aisles, df_departments, df_sample_submission = load_data(path_data)    

In [4]:
## Suggested by @Robert Goss https://www.kaggle.com/nickycan/lb-0-3805009-python-edition
import sys

def check_data(data):
    starting_size = sys.getsizeof(data)
    i = 0
    for c, dtype in zip(data.columns, data.dtypes):
        if 'int' in str(dtype):
            if min(data[c]) >=0:
                max_int =  max(data[c])
                if max_int <= 255:
                    data[c] = data[c].astype(np.uint8)
                elif max_int <= 65535:
                    data[c] = data[c].astype(np.uint16)
                elif max_int <= 4294967295:
                    data[c] = data[c].astype(np.uint32)
                i += 1
    print("Number of colums adjusted: {}\n".format(i))
    ## Changing known reorderd col to smaller int size
    data['reordered'] = np.nan_to_num(data['reordered']).astype(np.uint8)
    data.reordered.loc[data['reordered'] == 0] = np.nan  ## only replace such element in the column
    print("Reduced size {:.2%}".format(float(sys.getsizeof(data))/float(starting_size)))
    return data

In [5]:
my_check_list = [df_priors, df_train]
for this in my_check_list:
    check_data(this)
    

Number of colums adjusted: 4



/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Reduced size 166.67%
Number of colums adjusted: 4

Reduced size 166.67%


In [6]:
print(df_priors.shape)
print(df_train.shape)
print(df_orders.shape)
print(df_products.shape)
print(df_aisles.shape)
print(df_departments.shape)

(32434489, 4)
(1384617, 4)
(3421083, 7)
(49688, 4)
(134, 2)
(21, 2)


In [7]:
## df_priors, df_train, df_orders, df_products, df_aisles, df_departments, df_sample_submission
df_priors_table = df_priors.describe().applymap('{:.2f}'.format)
display(df_priors_table)
##display(df_data.iloc[:100, :9].describe().applymap('{:.2f}'.format))

,order_id,product_id,add_to_cart_order,reordered
count,32434489.00,32434489.00,32434489.00,19126536.00
mean,1710748.52,25576.34,8.35,1.00
std,987300.70,14096.69,7.13,0.00
min,2.00,1.00,1.00,1.00
25%,855943.00,13530.00,3.00,1.00
50%,1711048.00,25256.00,6.00,1.00
75%,2565514.00,37935.00,11.00,1.00
max,3421083.00,49688.00,145.00,1.00


In [8]:
print(df_priors_table.to_latex())

\begin{tabular}{lllll}
\toprule
{} &     order\_id &   product\_id & add\_to\_cart\_order &    reordered \\
\midrule
count &  32434489.00 &  32434489.00 &       32434489.00 &  19126536.00 \\
mean  &   1710748.52 &     25576.34 &              8.35 &         1.00 \\
std   &    987300.70 &     14096.69 &              7.13 &         0.00 \\
min   &         2.00 &         1.00 &              1.00 &         1.00 \\
25\%   &    855943.00 &     13530.00 &              3.00 &         1.00 \\
50\%   &   1711048.00 &     25256.00 &              6.00 &         1.00 \\
75\%   &   2565514.00 &     37935.00 &             11.00 &         1.00 \\
max   &   3421083.00 &     49688.00 &            145.00 &         1.00 \\
\bottomrule
\end{tabular}



In [9]:
display(df_train.describe().applymap('{:.2f}'.format))

,order_id,product_id,add_to_cart_order,reordered
count,1384617.00,1384617.00,1384617.00,828824.00
mean,1706297.62,25556.24,8.76,1.00
std,989732.65,14121.27,7.42,0.00
min,1.00,1.00,1.00,1.00
25%,843370.00,13380.00,3.00,1.00
50%,1701880.00,25298.00,7.00,1.00
75%,2568023.00,37940.00,12.00,1.00
max,3421070.00,49688.00,80.00,1.00


In [10]:
df_orders_table = df_orders.describe().applymap('{:.2f}'.format)
display(df_orders_table)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3421083.00,3421083.00,3421083.00,3421083.00,3421083.00,3214874.00
mean,1710542.00,102978.21,17.15,2.78,13.45,11.13
std,987581.74,59533.72,17.73,2.05,4.23,9.16
min,1.00,1.00,1.00,0.00,0.00,0.00
25%,855271.50,51394.00,5.00,1.00,10.00,4.00
50%,1710542.00,102689.00,11.00,3.00,13.00,7.00
75%,2565812.50,154385.00,23.00,5.00,16.00,15.00
max,3421083.00,206209.00,100.00,6.00,23.00,30.00


In [11]:
print(df_orders_table.to_latex())

\begin{tabular}{lllllll}
\toprule
{} &    order\_id &     user\_id & order\_number &   order\_dow & order\_hour\_of\_day & days\_since\_prior\_order \\
\midrule
count &  3421083.00 &  3421083.00 &   3421083.00 &  3421083.00 &        3421083.00 &             3214874.00 \\
mean  &  1710542.00 &   102978.21 &        17.15 &        2.78 &             13.45 &                  11.13 \\
std   &   987581.74 &    59533.72 &        17.73 &        2.05 &              4.23 &                   9.16 \\
min   &        1.00 &        1.00 &         1.00 &        0.00 &              0.00 &                   0.00 \\
25\%   &   855271.50 &    51394.00 &         5.00 &        1.00 &             10.00 &                   4.00 \\
50\%   &  1710542.00 &   102689.00 &        11.00 &        3.00 &             13.00 &                   7.00 \\
75\%   &  2565812.50 &   154385.00 &        23.00 &        5.00 &             16.00 &                  15.00 \\
max   &  3421083.00 &   206209.00 &       100.00 &        6

### The data is ready. ###

In [12]:
class setup_your_clock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

In [13]:
def ka_add_groupby_features_1_vs_n(df, group_columns_list, agg_dict, only_new_feature=True):
    '''Create statistical columns, group by group_columns_list 
       and compute stats on [N columns]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       agg_dict: python dictionary

       Return
       ------
       new pandas dataframe w/o original columns

       Example
       -------
       {real_column_name: {your_specified_new_column_name : method}}
       agg_dict = {'user_id':{'prod_tot_cnts':'count'},
                   'reordered':{'reorder_tot_cnts_of_this_prod':'sum'},
                   '_user_buy_product_times': {'_prod_order_once':lambda x: sum(x==1),
                                               '_prod_order_more_than_once':lambda x: sum(x==2)}}
       ka_add_stats_features_1_vs_n(train, ['product_id'], agg_dict)
    '''
    with setup_your_clock("add stats features"):
        try:
            if type(group_columns_list) == list:
                pass
            else:
                raise TypeError(group_columns_list + "should be a list")
        except TypeError as e:
            print(e)
            raise

        df_new = df.copy()

        ## step 1: group by
        grouped = df_new.groupby(group_columns_list)

        ## step 2: aggregation
        the_stats = grouped.agg(agg_dict)
        
        ##display(the_stats.head(5))
        the_stats.columns = the_stats.columns.droplevel(0)
        ##display(the_stats.head(5))
        
        the_stats.reset_index(inplace=True)
        ## determine whether we save only new features
        if only_new_feature:
            df_new = the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
    display(df_new.head(3))
    return df_new

## Product ##

In [14]:
display(df_orders.head(5))
print(df_orders.shape)

display(df_priors.head(5))
print(df_priors.shape)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


(3421083, 7)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1.0
1,2,28985,2,1.0
2,2,9327,3,NaN
3,2,45918,4,1.0
4,2,30035,5,NaN


(32434489, 4)


In [15]:
df_priors_orders_detail = df_orders.merge(right=df_priors, how='inner', on='order_id')
display(df_priors_orders_detail.head(5))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196,1,NaN
1,2539329,1,prior,1,2,8,NaN,14084,2,NaN
2,2539329,1,prior,1,2,8,NaN,12427,3,NaN
3,2539329,1,prior,1,2,8,NaN,26088,4,NaN
4,2539329,1,prior,1,2,8,NaN,26405,5,NaN


In [16]:
print(df_priors_orders_detail.shape)
#print(df_priors_orders_detail.columns)

(32434489, 10)


In [17]:
## create new variables
## _user_buy_product_times: user x buy y product ___ times

## https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html
## cumcount() + 1 for counting number of times

df_priors_orders_detail.loc[:,'_user_buy_product_times'] = \
  df_priors_orders_detail.groupby(['user_id', 'product_id']).cumcount() + 1
    
display(df_priors_orders_detail[:5])

# del df_priors_orders_detail['user_buy_product_times']
# display(df_priors_orders_detail.head(2))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,NaN,1
1,2539329,1,prior,1,2,8,NaN,14084,2,NaN,1
2,2539329,1,prior,1,2,8,NaN,12427,3,NaN,1
3,2539329,1,prior,1,2,8,NaN,26088,4,NaN,1
4,2539329,1,prior,1,2,8,NaN,26405,5,NaN,1


In [18]:
# ## simplify example

# ## _prod_tot_cnts: total counts of buying such product
# ## _prod_reorder_tot_cnts: total counts of re-ordering such product

# ## _prod_order_once: the product being ordered once
# ## _prod_order_more_than_once: the product being ordered more than once (lambda x: sum(x==2))
# ## Note: for being ordered 2 times or 10 times, it will be counted as one occurrence in _prod_order_more_than_once 
# agg_dict = {'user_id':{'_prod_tot_cnts':'count'},
#             'reordered':{'_prod_reorder_tot_cnts':'sum'}}

# mini_prd = ka_add_groupby_features_1_vs_n(df_priors_orders_detail[:10000], ['product_id'], agg_dict)
# display(mini_prd.head(5))

In [19]:
## _prod_tot_cnts: total counts of buying such product
## _prod_reorder_tot_cnts: total counts of re-ordering such product

## _prod_order_once: the product being ordered once
## _prod_order_more_than_once: the product being ordered more than once (lambda x: sum(x==2))
## Note: for being ordered 2 times or 10 times, it will be counted as one occurrence in _prod_order_more_than_once 
agg_dict = {'user_id':{'_prod_tot_cnts':'count'}, 
            'reordered':{'_prod_reorder_tot_cnts':'sum'}, 
            '_user_buy_product_times': {'_prod_buy_first_time_total_cnt':lambda x: sum(x==1),
                                        '_prod_buy_second_time_total_cnt':lambda x: sum(x==2)}}
df_prod = ka_add_groupby_features_1_vs_n(df_priors_orders_detail, ['product_id'], agg_dict)

add stats features begin ......


/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 493.75743675231934 s 



,product_id,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_buy_first_time_total_cnt,_prod_buy_second_time_total_cnt
0,1,1852,1136.0,716,276
1,2,90,12.0,78,8
2,3,277,203.0,74,36


In [20]:
## fillna zero for items only being purchased once
df_prod.loc[:, '_prod_reorder_tot_cnts'] = df_prod._prod_reorder_tot_cnts.fillna(0)

In [21]:
# _prod_reorder_prob: the_count_of_prod_buy_more_than_one / the_count_of_prod_buy_only_once
# _prod_reorder_ratio: for such product, reordered_count / total_count
df_prod['_prod_reorder_prob'] = df_prod._prod_buy_second_time_total_cnt / df_prod._prod_buy_first_time_total_cnt
df_prod['_prod_reorder_ratio'] = df_prod._prod_reorder_tot_cnts / df_prod._prod_tot_cnts
## df_prod['_prod_reorder_times'] = 1 + df_prod._prod_reorder_tot_cnts / df_prod._prod_buy_first_time_total_cnt

In [22]:
display(df_prod.head(5))

,product_id,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_buy_first_time_total_cnt,_prod_buy_second_time_total_cnt,_prod_reorder_prob,_prod_reorder_ratio
0,1,1852,1136.0,716,276,0.385475,0.613391
1,2,90,12.0,78,8,0.102564,0.133333
2,3,277,203.0,74,36,0.486486,0.732852
3,4,329,147.0,182,64,0.351648,0.446809
4,5,15,9.0,6,4,0.666667,0.600000


In [23]:
print(df_prod.isnull().values.any())
display(df_prod.loc[df_prod['_prod_reorder_tot_cnts'] == 0].head(5))

False


,product_id,_prod_tot_cnts,_prod_reorder_tot_cnts,_prod_buy_first_time_total_cnt,_prod_buy_second_time_total_cnt,_prod_reorder_prob,_prod_reorder_ratio
18,19,4,0.0,4,0,0.0,0.0
47,48,10,0.0,10,0,0.0,0.0
75,76,3,0.0,3,0,0.0,0.0
79,80,4,0.0,4,0,0.0,0.0
84,85,4,0.0,4,0,0.0,0.0


In [24]:
print(df_prod.head(2).to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
{} &  product\_id &  \_prod\_tot\_cnts &  \_prod\_reorder\_tot\_cnts &  \_prod\_buy\_first\_time\_total\_cnt &  \_prod\_buy\_second\_time\_total\_cnt &  \_prod\_reorder\_prob &  \_prod\_reorder\_ratio \\
\midrule
0 &           1 &            1852 &                  1136.0 &                             716 &                              276 &            0.385475 &             0.613391 \\
1 &           2 &              90 &                    12.0 &                              78 &                                8 &            0.102564 &             0.133333 \\
\bottomrule
\end{tabular}



## User ##

In [25]:
display(df_orders.head(5))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [26]:
## _user_total_orders: total order per user (max of order_number for such user)

## _user_sum_days_since_prior_order: sum of days_since_prior_order
## _user_mean_days_since_prior_order: average of days_since_prior_order

agg_dict_2 = {'order_number':{'_user_total_orders':'max'},
              'days_since_prior_order':{'_user_sum_days_since_prior_order':'sum', 
                                        '_user_mean_days_since_prior_order': 'mean'}}
df_users = ka_add_groupby_features_1_vs_n(df_orders[df_orders.eval_set == 'prior'], ['user_id'], agg_dict_2)

add stats features begin ......


/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 0.792248010635376 s 



,user_id,_user_total_orders,_user_sum_days_since_prior_order,_user_mean_days_since_prior_order
0,1,10,176.0,19.555555
1,2,14,198.0,15.230769
2,3,12,133.0,12.090909


In [27]:
display(df_priors_orders_detail.head(3))

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_user_buy_product_times
0,2539329,1,prior,1,2,8,NaN,196,1,NaN,1
1,2539329,1,prior,1,2,8,NaN,14084,2,NaN,1
2,2539329,1,prior,1,2,8,NaN,12427,3,NaN,1


In [28]:
## _user_reorder_ratio: reorder times / total order times
## _user_total_products: total number of purchased product
## _user_distinct_products: total number of unique purchased product
agg_dict_3 = {'reordered':
              {'_user_reorder_ratio': 
               lambda x: sum(df_priors_orders_detail.ix[x.index,'reordered']==1)/
                         sum(df_priors_orders_detail.ix[x.index,'order_number'] > 1)},
              'product_id':{'_user_total_products':'count', 
                            '_user_distinct_products': lambda x: x.nunique()}}
df_us = ka_add_groupby_features_1_vs_n(df_priors_orders_detail, ['user_id'], agg_dict_3)


add stats features begin ......


/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)
/Users/udothemath1984/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


add stats features end ......
time lapsing 1945.9306268692017 s 



,user_id,_user_reorder_ratio,_user_total_products,_user_distinct_products
0,1,0.759259,59,18
1,2,0.510989,195,102
2,3,0.705128,88,33


In [29]:
df_users = df_users.merge(df_us, how='inner')

In [30]:
## _user_average_basket: average product per order for each user

df_users['_user_average_basket'] = df_users._user_total_products / df_users._user_total_orders

## exclude prior --> train and test remain
df_us = df_orders[df_orders.eval_set != "prior"][['user_id', 'order_id', 'eval_set', 'days_since_prior_order']]

## rename days_since_prior_order -> time_since_last_order
df_us.rename(index=str, columns={'days_since_prior_order': 'time_since_last_order'}, inplace=True)

df_users = df_users.merge(df_us, how='inner')

In [31]:
print(df_users.head(2).to_latex())

\begin{tabular}{lrrrrrrrrrlr}
\toprule
{} &  user\_id &  \_user\_total\_orders &  \_user\_sum\_days\_since\_prior\_order &  \_user\_mean\_days\_since\_prior\_order &  \_user\_reorder\_ratio &  \_user\_total\_products &  \_user\_distinct\_products &  \_user\_average\_basket &  order\_id & eval\_set &  time\_since\_last\_order \\
\midrule
0 &        1 &                  10 &                             176.0 &                          19.555555 &             0.759259 &                    59 &                       18 &              5.900000 &   1187899 &    train &                   14.0 \\
1 &        2 &                  14 &                             198.0 &                          15.230769 &             0.510989 &                   195 &                      102 &             13.928571 &   1492625 &    train &                   30.0 \\
\bottomrule
\end{tabular}



## Detail of the order based on each user ##

In [32]:
# _up_order_count: order count of such product by each user
# _up_first_order_number: the order of such product from user's first time purchase
# _up_last_order_number:  the order of such product from user's last time purchase
# _up_average_cart_position: average add t ocart order
agg_dict_4 = {'order_number':{'_up_order_count': 'count', 
                              '_up_first_order_number': 'min', 
                              '_up_last_order_number':'max'}, 
              'add_to_cart_order':{'_up_average_cart_position': 'mean'}}

df_data = ka_add_groupby_features_1_vs_n(df=df_priors_orders_detail, 
                                      group_columns_list=['user_id', 'product_id'], 
                                      agg_dict=agg_dict_4)



add stats features begin ......


/Users/udothemath1984/anaconda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


add stats features end ......
time lapsing 41.581416845321655 s 



,user_id,product_id,_up_order_count,_up_first_order_number,_up_last_order_number,_up_average_cart_position
0,1,196,10,1,10,1.400000
1,1,10258,9,2,10,3.333333
2,1,10326,1,5,5,5.000000


In [33]:
# ## save file
# cstore_up = pd.HDFStore('saved_df_data.h5', mode='w')
# cstore_up.append('df_data', df_data)
# cstore_up.close()

### Save the files to .h5 format for furue retrieve ###

In [34]:
## save file

cstore = pd.HDFStore('saved_df_data.h5', mode='w')
cstore.append('df_data', df_data)
cstore.close()

cstore = pd.HDFStore('saved_df_prod.h5', mode='w')
cstore.append('df_prod', df_prod)
cstore.close()

cstore = pd.HDFStore('saved_df_users.h5', mode='w')
cstore.append('df_users', df_users)
cstore.close()


### Merging tables ###

In [35]:
df_data = df_data.merge(df_prod, how='inner', on='product_id').merge(df_users, how='inner', on='user_id')
## _up_order_rate: order product count/ order count
## _up_order_since_last_order: most recent order -last time of ordering such produect
## _up_order_rate_since_first_order: 
##    how many times buying such product / how time order you have placed since the first time you buy such product
df_data['_up_order_rate'] = df_data._up_order_count / df_data._user_total_orders
df_data['_up_order_since_last_order'] = df_data._user_total_orders - df_data._up_last_order_number
df_data['_up_order_rate_since_first_order'] = df_data._up_order_count / (df_data._user_total_orders - df_data._up_first_order_number + 1)

# add user_id to train set
df_train = df_train.merge(right=df_orders[['order_id', 'user_id']], how='left', on='order_id')
df_data = df_data.merge(df_train[['user_id', 'product_id', 'reordered']], on=['user_id', 'product_id'], how='left')

In [36]:
## release Memory
# del df_train, df_prod, df_users
# gc.collect()

In [37]:
df_data.dtypes

user_id                                 int64
product_id                             uint64
_up_order_count                         int64
_up_first_order_number                  int16
_up_last_order_number                   int16
_up_average_cart_position             float64
_prod_tot_cnts                          int64
_prod_reorder_tot_cnts                float64
_prod_buy_first_time_total_cnt          int64
_prod_buy_second_time_total_cnt         int64
_prod_reorder_prob                    float64
_prod_reorder_ratio                   float64
_user_total_orders                      int16
_user_sum_days_since_prior_order      float32
_user_mean_days_since_prior_order     float32
_user_reorder_ratio                   float64
_user_total_products                    int64
_user_distinct_products                uint16
_user_average_basket                  float64
order_id                                int32
eval_set                             category
time_since_last_order             

In [38]:
## access hdf5 file
## http://pandas.pydata.org/pandas-docs/version/0.20/io.html#io-hdf5

## save file
cstore = pd.HDFStore('saved_df_data_whole.h5', mode='w')
cstore.append('df_data', df_data)
## cstore.append('dfcat', dfcat, format='table', data_columns=['A'])
cstore.close()


In [39]:
# ## access to hdf5 file
# small_info = pd.read_hdf('saved_df_data.h5', 'df_data').head(5)
# ## pd.read_hdf('save_df_data.h5', 'table', where = ['index>2'])
# small_info

In [40]:
## Checking purpose
## prior, train, and test dataset have no overlap.
# order_ids_pri = df_priors.order_id.unique()
# order_ids_trn = df_train.order_id.unique()
# order_ids_tst = df_orders[df_orders.eval_set == 'test']['order_id'].unique()
# print(set(order_ids_pri).intersection(set(order_ids_trn)))
# print(set(order_ids_pri).intersection(set(order_ids_tst)))
# print(set(order_ids_trn).intersection(set(order_ids_tst)))